# SQL Murder Mystery

There's been a Murder in SQL City! The SQL Murder Mystery is designed to be both a self-directed lesson to learn SQL concepts and commands and a fun game for experienced SQL users to solve an intriguing crime.

<img src="./images/schema.png">

- First, we need to check the crime scene reports about the murder that happened

In [ ]:
SELECT *
FROM crime_scene_report
WHERE type = 'murder' AND city = 'SQL City'

<img src="./images/Screenshot 2026-02-05 092001.png">

### Key Details

- Victim:
    - Security Guard (took an arrow to the knee)

- Witnesses:
    - Annabel (lives on Franklin Ave)
    - Unknown (lives at the last house on Northwestern Dr)

- Now, we need to get more details about the witnesses so that we can get their interviews.

In [ ]:
SELECT *
FROM person
WHERE address_street_name = 'Franklin Ave'
    AND name LIKE 'Annabel%'

<img src="./images/Screenshot 2026-02-05 092351.png">

- Searching for Annabel's info is easy since all the information is given. However, the second witness is a tricky one since we don't know his/her name. But from the report, that witness lives at the last house of Northwestern Drive. Thus, we need to use an aggregating function

In [ ]:
SELECT *
FROM person
WHERE address_street_name = 'Northwestern Dr'
AND address_number = (
    SELECT MAX(address_number) 
    FROM person 
    WHERE address_street_name = 'Northwestern Dr'
);

<img src="./images/Screenshot 2026-02-05 093932.png">

- Since we know, their `id`, we can run a query to check the interviews of both witness

In [ ]:
SELECT *
FROM interview 
WHERE person_id IN (14887, 16371)

<img src="./images/Screenshot 2026-02-05 094614.png">

### Key Details

- Suspect works out at `Get Fit Now Gym`
- Membership number starts with `48Z` (`Gold Member`)
- Has a car with plate starts that includes `H42W`
- Annabel recognized the suspect on `January 9` (no year)

- Using multiple joins, we can create a query that uses the information above

In [ ]:
SELECT DISTINCT p.name
FROM person AS p
JOIN drivers_license AS dl
    ON p.license_id = dl.id
JOIN get_fit_now_member AS gfnm
    ON p.id = gfnm.person_id
JOIN get_fit_now_check_in AS gfnci
    ON gfnm.id = gfnci.membership_id
WHERE gfnm.membership_status = 'gold'
    AND gfnm.id LIKE '48Z%'
    AND gfnci.check_in_date = 20180109
    AND dl.plate_number LIKE '%H42W%'

<img src="./images/Screenshot 2026-02-05 102814.png">

- Only one name showed when all of the information was provided. Let's see if this is the guy we are looking for

<img src="./images/Screenshot 2026-02-05 103356.png">

- Now, let's see the transcript of the hitman to provide info on the mastermind

In [ ]:
SELECT i.transcript
FROM interview AS i
JOIN person AS p
    ON i.person_id = p.id
WHERE p.name = 'Jeremy Bowers'

<img src="./images/Screenshot 2026-02-05 103738.png">

### Key Details:

- `Woman` with a `lot of money`
- Height is around `5'5" (65")` to `5'7" (67")`
- `Red-haired`
- Drives a `Tesla Model S`
- Went to `SQL Symphony Concert` `3 times` in `December 2017`

- Since "lots of money" is subjective, I won't include this in the query.  

In [ ]:
SELECT DISTINCT p.name
FROM person AS p
JOIN drivers_license AS dl
	ON p.license_id = dl.id
JOIN facebook_event_checkin AS fec
	ON p.id = fec.person_id
WHERE dl.gender = 'female'
	AND dl.hair_color = 'red'
	AND dl.car_make = 'Tesla'
	AND dl.car_model = 'Model S'
	AND height BETWEEN 65 AND 67
	AND person_id IN (
		SELECT person_id
		FROM facebook_event_checkin
		WHERE date BETWEEN 20171200 AND 20171231
			AND event_name = 'SQL Symphony Concert'
		GROUP BY person_id
		HAVING COUNT(person_id) = 3
	)

<img src="./images/Screenshot 2026-02-05 112515.png">

- Let's see if we really got the mastermind of the crime

<img src="./images/Screenshot 2026-02-05 112616.png">